In [1]:
import os
import requests
import warnings
import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup as bs

warnings.filterwarnings('ignore')

In [2]:
myntra = pd.read_csv('/kaggle/input/myntra-168k-products/data.csv')
myntra.columns = [c.lstrip().rstrip().replace(' ', '') for c in myntra.columns]
myntra_pre = myntra[['product_name', 'product_link']]
myntra_pre.rename(columns={'product_name': 'description', 'product_link': 'product_url'}, inplace=True)
myntra_pre['description'] = myntra_pre['description'].apply(lambda x: x.lower().lstrip().rstrip())
myntra_pre['product_url'] = 'https://www.myntra.com/' + myntra_pre['product_url']

In [3]:
ajio = pd.read_csv('/kaggle/input/ajio-clothing-fashion/Ajio Fasion Clothing.csv',encoding='latin-1')
ajio.columns = [c.lstrip().rstrip().replace(' ', '') for c in ajio.columns]
ajio_pre = ajio[['Description', 'Product_URL']]
ajio_pre.rename(columns={'Description': 'description', 'Brand': 'brand_tag', 'Product_URL': 'product_url'}, inplace=True)
ajio_pre['description'] = ajio_pre['description'].apply(lambda x: x.lower().lstrip().rstrip())

In [4]:
flipkart = []
urls = {
    'https://www.flipkart.com/clothing-and-accessories/topwear/pr?sid=clo%2Cash&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DMen&otracker=nmenu_sub_Men_0_Top%20wear&' : 900, 
    'https://www.flipkart.com/clothing-and-accessories/bottomwear/pr?sid=clo%2Cvua&otracker=categorytree&p%5B%5D=facets.ideal_for%255B%255D%3DMen&otracker=nmenu_sub_Men_0_Bottom%20wear&' : 670,
    'https://www.flipkart.com/mens-footwear/pr?sid=osp,cil&otracker=nmenu_sub_Men_0_Footwear&' : 778
       }
for url, pages in urls.items():
    for page in tqdm(range(2, pages)):
      url = f'{url}&page={page}'
      res = requests.get(url)
      soup = bs(res.text, 'html.parser')
      for i, v in enumerate(soup.find_all('a', {'class': 'IRpwTa'})):
        flipkart.append([v.get_text(), 'www.flipkart.com' + v['href']])

flipkart = pd.DataFrame(flipkart)
flipkart.columns = ['description', 'product_url']
flipkart['description'] = flipkart['description'].apply(lambda x: x.lower().lstrip().rstrip())

100%|██████████| 776/776 [09:13<00:00,  1.40it/s]


In [5]:
data = myntra_pre.append(ajio_pre, ignore_index=True)
data = data.append(flipkart, ignore_index=True)
data = data.sample(frac = 1)


In [6]:
data.shape

(535321, 2)

In [7]:
data.to_csv('mercor_clothing_data.csv', index=False)